In [ ]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.manifold import TSNE
from sklearn.decomposition import FastICA

In [ ]:
df = pd.read_csv("./insurance.csv")
df.head()

,age,gender,bmi,children,smoker,region,insurance_cost
0,18,male,33.770,1,no,southeast,1725.55230
1,18,male,34.100,0,no,southeast,1137.01100
2,18,female,26.315,0,no,northeast,2198.18985
3,18,female,38.665,2,no,northeast,3393.35635
4,18,female,35.625,0,no,northeast,2211.13075


In [ ]:
df.tail()

,age,gender,bmi,children,smoker,region,insurance_cost
1333,64,female,31.825,2,no,northeast,16069.08475
1334,64,female,26.885,0,yes,northwest,29330.98315
1335,64,male,26.410,0,no,northeast,14394.55790
1336,64,male,36.960,2,yes,southeast,49577.66240
1337,64,male,23.760,0,yes,southeast,26926.51440


In [ ]:
df['region'].value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [ ]:
df.isna().sum()

age               0
gender            0
bmi               0
children          0
smoker            0
region            0
insurance_cost    0
dtype: int64

In [ ]:
df.describe()

,age,bmi,children,insurance_cost
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [ ]:
X = df.drop('insurance_cost', axis=1)
y = df['insurance_cost']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
numeric_features = ['age', 'bmi', 'children']
categorical_features = ['gender', 'smoker', 'region']

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
regressors = {
    'Random Forest': (RandomForestRegressor(), {'regressor__n_estimators': [10, 50, 100]}),
    'Linear Regression': (LinearRegression(), {}),
    'SVR': (SVR(), {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1, 10]})
}

In [ ]:
results = {}
for name, (regressor, params) in regressors.items():
    pipe = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', regressor)])
    grid_search = GridSearchCV(pipe, params, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {'best_model': best_model, 'mse': mse, 'mae': mae, 'r2': r2}

In [ ]:
results_df = pd.DataFrame(results).T
print(results_df)

                                                          best_model  \
Random Forest      (ColumnTransformer(transformers=[('num',\n    ...   
Linear Regression  (ColumnTransformer(transformers=[('num',\n    ...   
SVR                (ColumnTransformer(transformers=[('num',\n    ...   

                                mse          mae        r2  
Random Forest       19693165.270989  2519.368567  0.860578  
Linear Regression   36000021.710749  4307.778697  0.745131  
SVR                121162192.371116  5575.650037  0.142208  


Overall, Random Forest seems to be the most suitable regression algorithm for predicting insurance costs in this dataset, as it provides the most accurate predictions. Its ensemble nature allows it to handle complex relationships between features and the target variable effectively. Linear Regression, while simpler, can still provide reasonable results but may struggle with capturing non-linear relationships present in the data. SVR, although capable of handling non-linear data, didn't exhibit superior performance compared to Random Forest and Linear Regression in this specific scenario.

In [ ]:
regressors = {
    'Random Forest': (RandomForestRegressor(), {'regressor__n_estimators': [10, 50, 100]}),
    'Linear Regression': (LinearRegression(), {}),
    'SVR': (SVR(), {'regressor__kernel': ['linear', 'rbf'], 'regressor__C': [0.1, 1, 10]})
}

In [ ]:
dimensionality_reduction_methods = {
    't-SNE': TSNE(),
    'ICA': FastICA()
}

In [ ]:
results = {}
for name, (regressor, params) in regressors.items():
    for dr_name, dr_method in dimensionality_reduction_methods.items():
        if dr_name == 't-SNE':
            dr_method = 'passthrough'
        pipe = Pipeline(steps=[('preprocessor', preprocessor), ('reduce_dim', dr_method), ('regressor', regressor)])
        grid_search = GridSearchCV(pipe, params, cv=5, n_jobs=-1, error_score='raise')
        try:
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            y_pred = best_model.predict(X_test)
            mse = mean_squared_error(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            results[f'{name} with {dr_name}'] = {'best_model': best_model, 'mse': mse, 'mae': mae, 'r2': r2}
        except Exception as e:
            print(f"An error occurred for {name} with {dr_name}: {str(e)}")

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_fastica.py:542: FutureWarning: Starting in v1.3, whiten='unit-variance' will be used by default.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/dec

In [ ]:
results_df = pd.DataFrame(results).T
print(results_df)

                                                                     best_model  \
Random Forest with t-SNE      (ColumnTransformer(transformers=[('num',\n    ...   
Random Forest with ICA        (ColumnTransformer(transformers=[('num',\n    ...   
Linear Regression with t-SNE  (ColumnTransformer(transformers=[('num',\n    ...   
Linear Regression with ICA    (ColumnTransformer(transformers=[('num',\n    ...   
SVR with t-SNE                (ColumnTransformer(transformers=[('num',\n    ...   
SVR with ICA                  (ColumnTransformer(transformers=[('num',\n    ...   

                                           mse          mae        r2  
Random Forest with t-SNE        19715231.38995   2546.71986  0.860422  
Random Forest with ICA         24774616.264497  3046.429225  0.824603  
Linear Regression with t-SNE   36000021.710749  4307.778697  0.745131  
Linear Regression with ICA     35861701.407376  4268.054025   0.74611  
SVR with t-SNE                121162192.371116  5575.65003

Random Forest regression consistently outperformed Linear Regression and Support Vector Regression (SVR) across all dimensionality reduction techniques. It showcased lower mean squared error (MSE), mean absolute error (MAE), and higher R2 scores, indicating its superiority in capturing the relationships within the data.

The choice of dimensionality reduction method influenced model performance. t-SNE yielded better results compared to Independent Component Analysis (ICA) across all regression algorithms. This suggests that t-SNE was more effective in capturing the underlying structure of the data while reducing dimensionality, leading to improved model performance.

SVR performed poorly in this experiment, exhibiting significantly higher MSE and MAE values and negative R2 scores. This indicates that SVR struggled to effectively capture the relationships within the data and generalize to unseen samples. Further investigation and fine-tuning of SVR parameters may be necessary to improve its performance.

Based on these findings, employing Random Forest regression with t-SNE for dimensionality reduction would be the recommended approach for predicting insurance costs in this dataset. This combination consistently demonstrated the best performance metrics, providing more accurate predictions compared to other models and dimensionality reduction techniques.